In [106]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # see the value of multiple statements at once.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


python -m spacy download en_core_web_sm

In [107]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()

In [108]:
df = pd.read_csv('output.csv',header=None, names=['name', 'owner', 'description', 'topics']) 
df.columns = ['name', 'owner', 'description', 'topics']
df.shape
df.head()


(15864, 4)

,name,owner,description,topics
0,name,owner,description,topics
1,hub,mislav,A command-line tool that makes git easier to u...,[go homebrew git github-api pull-request]
2,lantern,getlantern,Lantern官方版本下载 蓝灯 翻墙 代理 科学上网 外网 加速器 梯子 路由 - Быс...,[lantern vpn censorship circumvention gfw acce...
3,revel,revel,"A high productivity, full-stack web framework ...",[web-framework web-application web-application...
4,direnv,direnv,unclutter your .profile,[direnv environment shell-extension bash zsh t...


In [109]:
df['description'] = df['description'] + ' ' + df['topics']

In [110]:
doc_set = df.values.T.tolist()[2]
print(doc_set[0:10])
# doc_set


['description topics', 'A command-line tool that makes git easier to use with GitHub. [go homebrew git github-api pull-request]', 'Lantern官方版本下载 蓝灯 翻墙 代理 科学上网 外网 加速器 梯子 路由 - Быстрый, надежный и безопасный доступ к открытому интернету - lantern proxy vpn censorship-circumvention censorship gfw accelerator پراکسی لنترن، ضدسانسور، امن، قابل اعتماد و پرسرعت [lantern vpn censorship circumvention gfw accelerator router]', 'A high productivity, full-stack web framework for the Go language. [web-framework web-application web-application-framework hacktoberfest revel]', 'unclutter your .profile [direnv environment shell-extension bash zsh tcsh fish shell]', 'An in-memory key:value store/cache (similar to Memcached) library for Go, suitable for single-machine applications. [go cache library]', 'DNS library in Go [dnssec go dns-library dns]', 'A Windows GUI toolkit for the Go Programming Language [go gui gui-toolkit win32 windows windows-desktop declarative declarative-ui]', 'Perkeep (née Camlist

In [111]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from math import isnan

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

tokenized_docs = []
for doc in doc_set:
    if type(doc) != str and isnan(doc):
        doc = ''
    tokens = tokenizer.tokenize(doc.lower())
    tokenized_docs.append(tokens)
    
print("Tokenized Docs:")
print(tokenized_docs[0:3])
# print(tokenized_docs)



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jbenitezg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Tokenized Docs:
[['description', 'topics'], ['a', 'command', 'line', 'tool', 'that', 'makes', 'git', 'easier', 'to', 'use', 'with', 'github', 'go', 'homebrew', 'git', 'github', 'api', 'pull', 'request'], ['lantern官方版本下载', '蓝灯', '翻墙', '代理', '科学上网', '外网', '加速器', '梯子', '路由', 'быстрый', 'надежный', 'и', 'безопасный', 'доступ', 'к', 'открытому', 'интернету', 'lantern', 'proxy', 'vpn', 'censorship', 'circumvention', 'censorship', 'gfw', 'accelerator', 'پراکسی', 'لنترن', 'ضدسانسور', 'امن', 'قابل', 'اعتماد', 'و', 'پرسرعت', 'lantern', 'vpn', 'censorship', 'circumvention', 'gfw', 'accelerator', 'router']]


In [112]:
lemmatized_tokens = []
for lst in tokenized_docs:
    tokens_lemma = [lemmatizer.lemmatize(i) for i in lst]
    lemmatized_tokens.append(tokens_lemma)
    
print(lemmatized_tokens[0:3])


[['description', 'topic'], ['a', 'command', 'line', 'tool', 'that', 'make', 'git', 'easier', 'to', 'use', 'with', 'github', 'go', 'homebrew', 'git', 'github', 'api', 'pull', 'request'], ['lantern官方版本下载', '蓝灯', '翻墙', '代理', '科学上网', '外网', '加速器', '梯子', '路由', 'быстрый', 'надежный', 'и', 'безопасный', 'доступ', 'к', 'открытому', 'интернету', 'lantern', 'proxy', 'vpn', 'censorship', 'circumvention', 'censorship', 'gfw', 'accelerator', 'پراکسی', 'لنترن', 'ضدسانسور', 'امن', 'قابل', 'اعتماد', 'و', 'پرسرعت', 'lantern', 'vpn', 'censorship', 'circumvention', 'gfw', 'accelerator', 'router']]


In [113]:
from stop_words import get_stop_words
en_stop_words = get_stop_words('en')

In [114]:
n=2
tokens = []
for lst in lemmatized_tokens:
    tokens.append([i for i in lst if not i in en_stop_words if len(i) > n])

print(tokens[0:3])
# tokens

[['description', 'topic'], ['command', 'line', 'tool', 'make', 'git', 'easier', 'use', 'github', 'homebrew', 'git', 'github', 'api', 'pull', 'request'], ['lantern官方版本下载', '科学上网', '加速器', 'быстрый', 'надежный', 'безопасный', 'доступ', 'открытому', 'интернету', 'lantern', 'proxy', 'vpn', 'censorship', 'circumvention', 'censorship', 'gfw', 'accelerator', 'پراکسی', 'لنترن', 'ضدسانسور', 'امن', 'قابل', 'اعتماد', 'پرسرعت', 'lantern', 'vpn', 'censorship', 'circumvention', 'gfw', 'accelerator', 'router']]


In [115]:
from gensim import corpora, models

dictionary = corpora.Dictionary(tokens)

In [116]:
corpus = [dictionary.doc2bow(text) for text in tokens]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [117]:
corpus[0]

[(0, 1), (1, 1)]

In [118]:
import gensim
ldamodel_3 = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
ldamodel_3.save('model3.gensim')
ldamodel_4 = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
ldamodel_4.save('model4.gensim')

In [119]:
for el in ldamodel_3.print_topics(num_topics=3, num_words=3):
    print(el,'\n')


(0, '0.040*"kubernetes" + 0.016*"cloud" + 0.015*"golang"') 

(1, '0.025*"golang" + 0.023*"minerproxy" + 0.019*"http"') 

(2, '0.053*"golang" + 0.011*"library" + 0.010*"data"') 


In [120]:
for el in ldamodel_4.print_topics(num_topics=3, num_words=3):
    print(el,'\n')


(0, '0.052*"golang" + 0.020*"server" + 0.016*"api"') 

(1, '0.030*"golang" + 0.024*"tool" + 0.022*"minerproxy"') 

(2, '0.040*"kubernetes" + 0.016*"cloud" + 0.015*"docker"') 


In [121]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')

In [122]:
corpus = pickle.load(open('corpus.pkl', 'rb'))

In [123]:
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')

In [124]:
import pyLDAvis.gensim

In [125]:
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)

/Users/jbenitezg/Documents/GitHub/ECI-MS-Thesis/topic-analysis/lda/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/jbenitezg/Documents/GitHub/ECI-MS-Thesis/topic-analysis/lda/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/jbenitezg/Documents/GitHub/ECI-MS-Thesis/topic-analysis/lda/.venv/lib/python3.12/site-packages/joblib/_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
/Users/jbenitezg/Documents/GitHub/ECI-MS-Thesis/topic-analysis/lda/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant ins

In [126]:
pyLDAvis.display(lda_display)